In [1]:
import sklearn
from sklearn.datasets import load_files

In [5]:
moviedir = r'.\movie_reviews'
# loading all files. 
movie = load_files(moviedir, shuffle=True)

print(movie.keys())
print(len(movie.data))

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])
2000


In [7]:
print(movie.filenames)

['.\\movie_reviews\\neg\\cv405_21868.txt'
 '.\\movie_reviews\\pos\\cv190_27052.txt'
 '.\\movie_reviews\\pos\\cv132_5618.txt' ...
 '.\\movie_reviews\\pos\\cv653_19583.txt'
 '.\\movie_reviews\\neg\\cv559_0057.txt'
 '.\\movie_reviews\\neg\\cv684_12727.txt']


In [22]:
print(movie.target_names)
print(movie.target)
print(movie.target.tolist().count(0))
print(movie.target.tolist().count(1))

['neg', 'pos']
[0 1 1 ... 1 0 0]
1000
1000


In [12]:
print(movie.data[1])

b"good films are hard to find these days . \ngreat films are beyond rare . \nproof of life , russell crowe's one-two punch of a deft kidnap and rescue thriller , is one of those rare gems . \na taut drama laced with strong and subtle acting , an intelligent script , and masterful directing , together it delivers something virtually unheard of in the film industry these days , genuine motivation in a story that rings true . \nconsider the strange coincidence of russell crowe's character in proof of life making the moves on a distraught wife played by meg ryan's character in the film -- all while the real russell crowe was hitching up with married woman meg ryan in the outside world . \ni haven't seen this much chemistry between actors since mcqueen and mcgraw teamed up in peckinpah's masterpiece , the getaway . \nbut enough with the gossip , let's get to the review . \nthe film revolves around the kidnapping of peter bowman ( david morse ) , an american engineer working in south america

# A detour: try out CountVectorizer & TF-IDF
extractig features from text data per sample

In [27]:
from sklearn.feature_extraction.text import CountVectorizer
import nltk

In [28]:
# Turn off pretty printing of jupyter notebook... it generates long lines
%pprint

Pretty printing has been turned OFF


In [29]:
# Example: let's ake Three tiny "documents"
docs = ['A rose is a rose is a rose is a rose.',
        'Oh, what a fine day it is.',
        "A day ain't over till it's truly over."]

In [38]:
import nltk
nltk.download('punkt')
# Initialize a CountVectorizer to use NLTK's tokenizer instead of its 
#    default one (which ignores punctuation and stopwords). 
# Minimum document frequency set to 1. 
fooVzer = CountVectorizer(min_df=1, tokenizer=nltk.word_tokenize)
# (1) fit: adapts fooVzer to the supplied text data (rounds up top words into vector space)
docs_counts = fooVzer.fit(docs)
# fooVzer now contains vocab dictionary which maps unique words to indexes
print(len(fooVzer.vocabulary_))
print(fooVzer.vocabulary_.keys())
print(fooVzer.vocabulary_)

16
dict_keys(['a', 'rose', 'is', '.', 'oh', ',', 'what', 'fine', 'day', 'it', 'ai', "n't", 'over', 'till', "'s", 'truly'])
{'a': 3, 'rose': 12, 'is': 7, '.': 2, 'oh': 10, ',': 1, 'what': 15, 'fine': 6, 'day': 5, 'it': 8, 'ai': 4, "n't": 9, 'over': 11, 'till': 13, "'s": 0, 'truly': 14}


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sales2\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [42]:
 # transform: creates and returns a count-vectorized output of docs
docs_counts = fooVzer.transform(docs)
print(docs_counts.shape)
# this vector is small enough to view in a full, non-sparse form! 
print(docs_counts.toarray())
print(docs_counts[0])

(3, 16)
[[0 0 1 4 0 0 0 3 0 0 0 0 4 0 0 0]
 [0 1 1 1 0 1 1 1 1 0 1 0 0 0 0 1]
 [1 0 1 1 1 1 0 0 1 1 0 2 0 1 1 0]]
  (0, 2)	1
  (0, 3)	4
  (0, 7)	3
  (0, 12)	4


In [43]:
# Convert raw frequency counts into TF-IDF (Term Frequency -- Inverse Document Frequency) values
from sklearn.feature_extraction.text import TfidfTransformer
fooTfmer = TfidfTransformer()
# Again, fit and transform
docs_tfidf = fooTfmer.fit_transform(docs_counts)

In [44]:
# TF-IDF values
# raw counts have been normalized against document length, 
# terms that are found across many docs are weighted down ('a' vs. 'rose')
print(docs_tfidf.shape)
print(docs_tfidf.toarray())

(3, 16)
[[0.         0.         0.11337964 0.45351858 0.         0.
  0.         0.4379908  0.         0.         0.         0.
  0.7678737  0.         0.         0.        ]
 [0.         0.39427404 0.2328646  0.2328646  0.         0.29985557
  0.39427404 0.29985557 0.29985557 0.         0.39427404 0.
  0.         0.         0.         0.39427404]
 [0.30352608 0.         0.17926739 0.17926739 0.30352608 0.23083941
  0.         0.         0.23083941 0.30352608 0.         0.60705216
  0.         0.30352608 0.30352608 0.        ]]


In [45]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidVectorizer = TfidfVectorizer()
docs_counts_simple = tfidVectorizer.fit_transform(docs)
print(docs_counts_simple.toarray())
print(tfidVectorizer.get_feature_names())

[[0.         0.         0.         0.49546155 0.         0.
  0.         0.86862987 0.         0.         0.        ]
 [0.         0.34949812 0.45954803 0.34949812 0.34949812 0.45954803
  0.         0.         0.         0.         0.45954803]
 [0.35013871 0.26628951 0.         0.         0.26628951 0.
  0.70027741 0.         0.35013871 0.35013871 0.        ]]
['ain', 'day', 'fine', 'is', 'it', 'oh', 'over', 'rose', 'till', 'truly', 'what']


# Back to real data: movie reviews

In [46]:
# Split data into training and test sets
from sklearn.model_selection import train_test_split
docs_train, docs_test, y_train, y_test = train_test_split(movie.data, movie.target, 
                                                          test_size = 0.20, random_state = 12)

In [47]:
# initialize CountVectorizer
movieVzer= CountVectorizer(min_df=2, tokenizer=nltk.word_tokenize, max_features=3000) # use top 3000 words only. 78.25% acc.
# movieVzer = CountVectorizer(min_df=2, tokenizer=nltk.word_tokenize)         # use all 25K words. Higher accuracy

# fit and tranform using training text 
docs_train_counts = movieVzer.fit_transform(docs_train)

In [48]:
# 'screen' is found in the corpus, mapped to index 2290
print(len(movieVzer.vocabulary_))
print(movieVzer.vocabulary_)

3000
{'part': 1942, 'buddy': 398, 'comedy': 547, ',': 17, 'story': 2511, 'nature': 1802, 'tale': 2602, 'meet': 1689, 'the': 2648, 'is': 1415, 'not': 1840, 'nearly': 1804, 'as': 219, 'interesting': 1396, 'any': 180, 'of': 1872, 'those': 2676, '.': 21, 'in': 1356, 'fact': 952, 'it': 1419, 'an': 155, 'you': 2994, 'to': 2703, 'phil': 1981, 'and': 158, '(': 13, 'paul': 1958, 'steve': 2497, 'van': 2823, ')': 14, 'are': 204, 'twin': 2773, 'famous': 973, 'eric': 889, 'wants': 2868, 'his': 1280, 'be': 283, 'perfect': 1963, 'fortune': 1084, 'instead': 1383, 'two': 2777, 'set': 2334, 'them': 2654, 'straight': 2514, 'he': 1242, 'into': 1399, 'a': 64, 'camp': 426, 'which': 2917, 'pair': 1932, 'go': 1164, 'after': 113, 'several': 2341, 'where': 2915, 'discover': 763, 'there': 2660, "'s": 10, 'no': 1830, 'brothers': 392, 'upon': 2808, 'routine': 2240, 'identity': 1340, 'plot': 2014, 'they': 2663, 'arrive': 212, 'at': 231, 'national': 1799, 'park': 1939, 'new': 1818, 'rather': 2132, 'than': 2644, 'bac

In [49]:
print(len(movieVzer.stop_words_))
print(movieVzer.stop_words_)

38765
{'push', 'robber', 'soprano', 'recalling', 'busses', 'hiroshima', 'kaminski', 'volume', 'snitches', "'leon", 'letterbox', 'meticulous', 'coinciding', 'melbourne', 'orient', 'europe', 'subdue', 'kari', 'theft', 'schreber', 'i=92ve', 'haynes', 'woo-like', 'boyish', 'saddam', 'vulcan', 'stranglers', 'double-lines', 'harbinger', 'tunie', 'glimpsed', 'coruscant', 'kressler', 'transportation', 'intellectuals', 'attached', 'gangsta', 'done-that', 'ably', 'listing', 'posse', 'splendour', 'sigourney', 'writings', 'phils', 'pseudo-world', 'sweetheart', 'triple', 'consummate', 'real-time', 'loren', 'benjamin', 'bumps', 'radical', 'one-upping', 'streamline', 'grandson', 'matchup', 'promoters', 'amid', 'hitherto', 'hemant', 'big-', 'refused', 'overpraised', 'outrageously', 'flunked', 'abel', 'cover-ups', 'vibrate', 'newswoman', 'lender', 'echo', 'hurled', 'stifling', 'critic/david', 'schwartznager', 'dissatisfaction', 'tentative', 'hails', 'inability', 'nether', 'moronic', 'sponsor', 'planing

In [50]:
# huge dimensions! 1,600 documents, 3K unique terms. 
print(docs_train_counts.shape)

(1600, 3000)


In [51]:
# Convert raw frequency counts into TF-IDF values
movieTfmer = TfidfTransformer()
docs_train_tfidf = movieTfmer.fit_transform(docs_train_counts)

In [55]:
print(docs_train_tfidf[0])

  (0, 2994)	0.03171983006661036
  (0, 2977)	0.018670754955934604
  (0, 2945)	0.06257864563823713
  (0, 2920)	0.013766405072407823
  (0, 2917)	0.05121105748516787
  (0, 2915)	0.043069188429219356
  (0, 2913)	0.01601522450648827
  (0, 2897)	0.05329022537394579
  (0, 2887)	0.042163562921878166
  (0, 2885)	0.041360713004590585
  (0, 2868)	0.032689295068791015
  (0, 2823)	0.048722101935416894
  (0, 2808)	0.03767355544981579
  (0, 2807)	0.01641776488933369
  (0, 2793)	0.06787499941791177
  (0, 2779)	0.044511586587694316
  (0, 2777)	0.06118044423921711
  (0, 2773)	0.06068000807590912
  (0, 2714)	0.03767355544981579
  (0, 2703)	0.15859020006482663
  (0, 2676)	0.02506377564430205
  (0, 2673)	0.012768578190847493
  (0, 2663)	0.047303010718078096
  (0, 2660)	0.031413118380494834
  (0, 2654)	0.0599047025215529
  :	:
  (0, 241)	0.026331279795350132
  (0, 231)	0.014073515234775652
  (0, 223)	0.04967258010743341
  (0, 219)	0.025393865523434124
  (0, 212)	0.058800660212652234
  (0, 204)	0.067418812264

In [56]:
# Same dimensions, now with tf-idf values instead of raw frequency counts
docs_train_tfidf.shape

(1600, 3000)

# The feature extraction functions and traning data are ready.
Vectorizer and transformer have been built from the training data
Training data text was also turned into TF-IDF vector form
Next up: test data
You have to prepare the test data using the same feature extraction scheme

In [57]:
# Using the fitted vectorizer and transformer, tranform the test data
docs_test_counts = movieVzer.transform(docs_test)
docs_test_tfidf = movieTfmer.transform(docs_test_counts)

# Training and testing a Naive Bayes classifier

In [58]:
# Now ready to build a classifier. 
# We will use Multinominal Naive Bayes as our model
from sklearn.naive_bayes import MultinomialNB

In [59]:
# Train a Multimoda Naive Bayes classifier. Again, we call it "fitting"
clf = MultinomialNB()
clf.fit(docs_train_tfidf, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [60]:
# Predict the Test set results, find accuracy
y_pred = clf.predict(docs_test_tfidf)
sklearn.metrics.accuracy_score(y_test, y_pred)

0.7875

In [61]:
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[166,  40],
       [ 45, 149]], dtype=int64)

# Trying the classifier on fake movie reviews

In [62]:
# very short and fake movie reviews
reviews_new = ['This movie was excellent', 'Absolute joy ride', 
            'Steven Seagal was terrible', 'Steven Seagal shone through.', 
              'This was certainly a movie', 'Two thumbs up', 'I fell asleep halfway through', 
              "We can't wait for the sequel!!", '!', '?', 'I cannot recommend this highly enough', 
              'instant classic.', 'Steven Seagal was amazing. His performance was Oscar-worthy.',
              'On the whole, Enthavaralaina is a crime horror thriller which fails to create any excitement for the audience. Too many unnecessary scenes, forced emotions, and disappointing narration makes the film go down in no time and ends as a painful watch this weekend.',
              'On the whole, Maharshi is the journey of a man who finds his roots back in his motherland. Who better than Mahesh Babu to portray all this which has been narrated in an emotional manner by Vamsi Paidipally. Top-notch performances, decent humor, relatable social message, and Mahesh’s screen presence are main assets. If you ignore the slow pace and slightly predictable second half, this film has in it to work with a major section of the audience and the holiday summer season will help the film in a big way.']

reviews_new_counts = movieVzer.transform(reviews_new)         # turn text into count vector
reviews_new_tfidf = movieTfmer.transform(reviews_new_counts)  # turn into tfidf vector

In [63]:
# have classifier make a prediction
pred = clf.predict(reviews_new_tfidf)

In [64]:
# print out results
for review, category in zip(reviews_new, pred):
    print('%r => %s' % (review, movie.target_names[category]))

'This movie was excellent' => pos
'Absolute joy ride' => pos
'Steven Seagal was terrible' => neg
'Steven Seagal shone through.' => neg
'This was certainly a movie' => neg
'Two thumbs up' => neg
'I fell asleep halfway through' => neg
"We can't wait for the sequel!!" => neg
'!' => neg
'?' => neg
'I cannot recommend this highly enough' => pos
'instant classic.' => pos
'Steven Seagal was amazing. His performance was Oscar-worthy.' => neg
'On the whole, Enthavaralaina is a crime horror thriller which fails to create any excitement for the audience. Too many unnecessary scenes, forced emotions, and disappointing narration makes the film go down in no time and ends as a painful watch this weekend.' => neg
'On the whole, Maharshi is the journey of a man who finds his roots back in his motherland. Who better than Mahesh Babu to portray all this which has been narrated in an emotional manner by Vamsi Paidipally. Top-notch performances, decent humor, relatable social message, and Mahesh’s screen 